# EFASv5 - extraer caudal
***

***Autor:** Jesús Casado Rodríguez*<br>
***Fecha:** 23-09-2023*<br>

**Introducción:**<br>
Este _notebook_ extrae las series temporales de caudal simulado por el reanálisis de EFASv5 en los puntos/estaciones de interés.

Si se define `estaciones = 'ROEA'`, se carga una capa de puntos con las estaciones de aforo de la Red Oficial de Estaciones de Aforo seleccionadas en el notebook *1_Anuario_seleccion_estaciones.ipynb*. En esta capa GIS incluye las coordenadas corregidas para que los puntos se ubiquen sobre lo que EFASv5 considera río.

Si se define `estaciones = 'EFAS'`, se carga una capa de puntas con los puntos de reporte del sistema EFAS.

In [6]:
import os
os.environ['USE_PYGEOS'] = '0'
import glob
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import geopandas as gpd
# from shapely.geometry import Point
from tqdm.notebook import tqdm
import yaml
from pathlib import Path

## Configuración

In [29]:
with open("config.yml", "r", encoding='utf8') as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)

# rutas de entrada y salida de datos
path_anuario = Path(cfg['rutas']['anuario_aforos']['repo'])
path_efas = Path(cfg['rutas']['EFAS'].get('repo', '../../data/EFAS'))
path_out = path_out / 'caudal'
for path in [path_out]:
    if path.exists() is False:
        path.mkdir()

Definir si se van a extraer las series para los puntos del *Anuario de Aforos* (ROEA) o para los puntos de EFAS.

In [43]:
estaciones = 'ROEA' #'EFAS'

## Estaciones
### Estaciones ROEA

In [44]:
if estaciones == 'ROEA':
    
    # cargar shapefile de puntos
    puntos = gpd.read_file(path_anuario / 'GIS/estaciones_seleccion_LISFLOOD.shp')
    puntos.set_index('gauge_id', drop=True, inplace=True)

    # xarrays con las coordenadas de las estaciones
    x = xr.DataArray(puntos.geometry.x, dims='id')
    y = xr.DataArray(puntos.geometry.y, dims='id')

### Puntos de EFAS

In [34]:
if estaciones == 'EFAS':
    
    # cargar shapefile de puntos
    puntos = gpd.read_file(path_efas / 'GIS/reporting_points.shp')
    # puntos.geometry = gpd.points_from_xy(puntos.LisfloodX, puntos.LisfloodY)
    puntos.set_index('ObsID', drop=True, inplace=True)

    # xarrays con las coordenadas de las estaciones
    x = xr.DataArray(puntos.geometry.x, dims='id')
    y = xr.DataArray(puntos.geometry.y, dims='id')

## Caudal
### Reanálisis EFAS

In [ ]:
folder = {'EFAS': 'reporting_points',
          'ROEA': 'estaciones'}

# extreaer series de caudal archivo a archivo
for file in tqdm(glob.glob(f'{path_efas}/reanalysis/dis_*.nc')):

    # check if output file already exists
    file_out = '{0}/{1}/{2}'.format(path_out, folder[estaciones], file.split('\\')[-1])
    if os.path.exists(file_out):
        continue
    
    # load data
    dis = xr.open_dataset(file, chunks='auto')['dis']
    dis.close()
    
    # extract discharge for the selected stations
    dis = dis.sel(lon=x, lat=y, method='nearest')
    dis = dis.drop(['lon', 'lat'])
    
    # resample to daily timesteps. WARNING! The data is shifted 6 h!!!
    #dis['time'] = dis.time + np.timedelta64(6, 'h')
    dis = dis.resample(time='1D').mean()
        
    # compute the lazy DataArray
    dis = dis.compute()

    # export
    dis.name = 'discharge'
    dis.to_netcdf(file_out)